In [36]:
import orjson as json
import datetime
from collections import Counter
from tqdm import tqdm
from pprint import pprint

In [9]:
file = open("submissions/RS_2023-09", "r")

# Parameters to research about:
dict = {
    "subreddit": Counter(),
    "id": Counter(),
    "author": Counter(),
    "post_hint": Counter(),
    "score": Counter(),
    "is_original_content": Counter(),
    "likes": Counter(),
    "media": Counter(),
    "over_18": Counter(),
    "is_video": Counter(),
    "subreddit_type": Counter()
}
print(dict)

{'subreddit': Counter(), 'id': Counter(), 'author': Counter(), 'post_hint': Counter(), 'score': Counter(), 'is_original_content': Counter(), 'likes': Counter(), 'media': Counter(), 'over_18': Counter(), 'is_video': Counter(), 'subreddit_type': Counter()}


In [10]:
output = open("submissions/RS_2023-09-clean", "w+")
for line in tqdm(file):
    try:
        data = json.loads(line)
        
        # filter 1: add to new_data if not over_18 and not is_video
        if str(data['over_18']) == "False" and str(data['is_video']) == "False":
            output.write(json.dumps(data).decode("utf8") + "\n")
        
        # update each key's counter for the current data
        for key in dict.keys():
            if key in data:
                dict[key].update([str(data[key])])
    except Exception as e:
        print("ERROR: ", e)
        continue

output.close()
file.close()

0it [00:00, ?it/s]

3658994it [01:48, 33869.68it/s]

ERROR:  unexpected end of data: line 1 column 5442 (char 5441)


#### Counts of the data without any filters:

In [11]:
# print out the results
for key in dict.keys():
    print(key, len(dict[key]))
    print(dict[key].most_common(10))
    print()

subreddit 195221
[('dirtyr4r', 21759), ('Starfield', 20846), ('jerkbudsNolimitssss0', 20596), ('jerkbuds_titi', 19596), ('AskReddit', 16733), ('GaySnapchatImages', 15758), ('SnapchatSexting4you', 13444), ('DisneyNewsfeed', 13443), ('GaySnapchatShare', 12287), ('DirtySnapchat', 12167)]

id 3658993
[('166rg7m', 1), ('166rg7n', 1), ('166rg7o', 1), ('166rg7r', 1), ('166rg7s', 1), ('166rg7t', 1), ('166rg7u', 1), ('166rg7v', 1), ('166rg7w', 1), ('166rg7x', 1)]

author 1045697
[('AutoModerator', 32425), ('[deleted]', 20327), ('EchoJobs', 13869), ('LifeCracker', 13443), ('PersonalizedGameRecs', 11303), ('bsgqlocasdfd', 7175), ('Appzxt', 6121), ('AutoNewsAdmin', 5229), ('enre19', 5078), ('Own_Laugh6357', 4953)]

post_hint 7
[('image', 1124544), ('link', 382599), ('rich:video', 230692), ('self', 92924), ('hosted:video', 87113), ('video', 26), ('gallery', 4)]

score 1518
[('1', 3581000), ('2', 21595), ('3', 9447), ('4', 4392), ('5', 4118), ('6', 3159), ('7', 2480), ('8', 1951), ('9', 1597), ('10'

In [31]:
file = open("submissions/RS_2023-09-clean", "r")
output = open("submissions/RS_2023-09-clean2", "w+")
dict = {
    "subreddit": Counter(),
    "id": Counter(),
    "author": Counter(),
    "post_hint": Counter(),
    "score": Counter(),
    "is_original_content": Counter(),
    "likes": Counter(),
    "media": Counter(),
    "over_18": Counter(),
    "is_video": Counter(),
    "subreddit_type": Counter()
}
    
foundError = False
prevData = None
for line in tqdm(file):
    if foundError:
        print("prevData: ", prevData)
        break
    
    try:
        data = json.loads(line)
        
        # filter 2: remove galleries and rich video from the data
        if ('is_gallery' in data and str(data['is_gallery']) == "True") or \
           ('post_hint' in data and str(data['post_hint']) == 'rich:video'):
            pass
        else:
            # filter 3: update image
            if 'post_hint' in data and str(data['post_hint']) == "image" and 'preview' in data:
                # handle the cases where either resolutions or source or both is empty
                images = data['preview']['images'][0]
                if len(images['resolutions']) == 0:
                    if len(images['source']) == 0:
                        continue
                    else:
                        data['image'] = images['source']['url']
                        data.pop('preview', None)
                        output.write(json.dumps(data).decode("utf8") + "\n")
                        continue
                
                # variables to find the best resolution
                preferredResolution = 90000
                resolutions = images['resolutions']
                minDiff = abs((resolutions[0]['width'] * resolutions[0]['height']) - preferredResolution)
                img = resolutions[0]['url']
                
                # find the resolution closest to 90,000 pixels
                for res in resolutions:
                    diff = abs((res['width'] * res['height']) - preferredResolution)
                    if diff < minDiff:
                        minDiff = diff
                        img = res['url']
                data['image'] = img
                data.pop('preview', None)
                
            output.write(json.dumps(data).decode("utf8") + "\n")
        
        # update each key's counter for the current data
        for key in dict.keys():
            if key in data:
                dict[key].update([str(data[key])])
    except Exception as e:
        print("ERROR: ", e)
        foundError = True
        prevData = data
        continue
    
file.close()
output.close()

1763351it [00:52, 33297.11it/s]


#### Counts of the data with filters:
* over_18 = False
* is_video = False

In [32]:
# print out the results
for key in dict.keys():
    print(key, len(dict[key]))
    print(dict[key].most_common(10))
    print()

subreddit 126754
[('Starfield', 19957), ('jerkbudsNolimitssss0', 16642), ('AskReddit', 15777), ('DisneyNewsfeed', 13443), ('PersonalizedGameRecs', 11303), ('PokemonGoRaids', 8616), ('relationship_advice', 7341), ('cuckik_leakkers', 6404), ('u_Appzxt', 6120), ('teenagers', 5572)]

id 1763198
[('166rg7o', 1), ('166rg7r', 1), ('166rg7s', 1), ('166rg7u', 1), ('166rg7y', 1), ('166rg7z', 1), ('166rg80', 1), ('166rg83', 1), ('166rg84', 1), ('166rg89', 1)]

author 792889
[('AutoModerator', 30873), ('EchoJobs', 13869), ('LifeCracker', 13443), ('PersonalizedGameRecs', 11303), ('[deleted]', 7535), ('bsgqlocasdfd', 6338), ('Appzxt', 6120), ('AutoNewsAdmin', 5225), ('enre19', 5076), ('piealamodelemonade', 4735)]

post_hint 5
[('image', 348824), ('link', 235708), ('self', 83845), ('rich:video', 61707), ('gallery', 4)]

score 1241
[('1', 1725844), ('2', 9429), ('3', 4159), ('4', 2345), ('5', 2006), ('6', 1527), ('7', 1166), ('8', 942), ('9', 807), ('10', 667)]

is_original_content 2
[('False', 175636

In [46]:
file = open("submissions/RS_2023-09-clean2", "r")
output = open("submissions/RS_2023-09-clean3", "w+")
dict = {
    "subreddit": Counter(),
    "id": Counter(),
    "author": Counter(),
    "score": Counter(),
    "title": Counter(),
    "date": Counter(),
    "selftext": Counter(),
    "image": Counter(),
    "spoiler": Counter()
}
    
foundError = False
prevData = None
for line in tqdm(file):
    if foundError:
        print("prevData: ", prevData)
        break
    
    try:
        data = json.loads(line)
        
        # convert utc time to date
        data['date'] = datetime.datetime.fromtimestamp(data['created_utc']).strftime('%Y-%m-%d')
        
        # remove all elements that are not in the dict and write to file
        new_data = {}
        for key in dict.keys():
            if key in data:
                new_data[key] = data[key]
        output.write(json.dumps(new_data).decode("utf8") + "\n")
        
        data.pop('date', None)
        
        # update each key's counter for the current data
        for key in dict.keys():
            if key in data:
                dict[key].update([str(data[key])])
    except Exception as e:
        print("ERROR: ", e)
        foundError = True
        prevData = data
        continue
    
file.close()
output.close()

1575821it [00:37, 41621.28it/s]


#### Counts of the data with filters:
* over_18 = False
* is_video = False
* is_gallery = False
* remove "rich:video"
* replace "preview" with "image"

In [47]:
# print out the results
for key in dict.keys():
    print(key, len(dict[key]))
    print(dict[key].most_common(10))
    print()

subreddit 116992
[('Starfield', 18999), ('jerkbudsNolimitssss0', 15872), ('AskReddit', 15777), ('DisneyNewsfeed', 13443), ('PersonalizedGameRecs', 11303), ('PokemonGoRaids', 8593), ('relationship_advice', 7341), ('u_Appzxt', 6120), ('cuckik_leakkers', 5899), ('teenagers', 5556)]

id 1575821
[('166rg7o', 1), ('166rg7r', 1), ('166rg7u', 1), ('166rg7y', 1), ('166rg7z', 1), ('166rg80', 1), ('166rg83', 1), ('166rg84', 1), ('166rg89', 1), ('166rg8a', 1)]

author 716065
[('AutoModerator', 28263), ('EchoJobs', 13869), ('LifeCracker', 13443), ('PersonalizedGameRecs', 11303), ('[deleted]', 6894), ('bsgqlocasdfd', 6338), ('Appzxt', 6120), ('AutoNewsAdmin', 5225), ('enre19', 5076), ('piealamodelemonade', 4735)]

score 1156
[('1', 1543648), ('2', 8471), ('3', 3723), ('4', 2099), ('5', 1784), ('6', 1327), ('7', 988), ('8', 805), ('9', 670), ('10', 556)]

title 1176538
[('Dale Wilson', 3445), ('Aria Electra Video Baby Alien Fan Bus Leaked The Fan Van Twitter', 2742), ('Praise Todd Howard', 2626), ('V

In [48]:
file = open("submissions/RS_2023-09-clean3", "r")
# output = open("submissions/RS_2023-09-clean4", "w+")
dict = {
    "subreddit": Counter(),
    "id": Counter(),
    "author": Counter(),
    "score": Counter(),
    "title": Counter(),
    "date": Counter(),
    "selftext": Counter(),
    "image": Counter(),
    "spoiler": Counter()
}
    
foundError = False
prevData = None
for line in tqdm(file):
    if foundError:
        print("prevData: ", prevData)
        break
    
    try:
        data = json.loads(line)
        
        # update each key's counter for the current data
        for key in dict.keys():
            if key in data:
                dict[key].update([str(data[key])])
    except Exception as e:
        print("ERROR: ", e)
        foundError = True
        prevData = data
        continue
    
file.close()
# output.close()

1575821it [00:13, 113111.49it/s]


In [49]:
# print out the results
for key in dict.keys():
    print(key, len(dict[key]))
    print(dict[key].most_common(10))
    print()

subreddit 116992
[('Starfield', 18999), ('jerkbudsNolimitssss0', 15872), ('AskReddit', 15777), ('DisneyNewsfeed', 13443), ('PersonalizedGameRecs', 11303), ('PokemonGoRaids', 8593), ('relationship_advice', 7341), ('u_Appzxt', 6120), ('cuckik_leakkers', 5899), ('teenagers', 5556)]

id 1575821
[('166rg7o', 1), ('166rg7r', 1), ('166rg7u', 1), ('166rg7y', 1), ('166rg7z', 1), ('166rg80', 1), ('166rg83', 1), ('166rg84', 1), ('166rg89', 1), ('166rg8a', 1)]

author 716065
[('AutoModerator', 28263), ('EchoJobs', 13869), ('LifeCracker', 13443), ('PersonalizedGameRecs', 11303), ('[deleted]', 6894), ('bsgqlocasdfd', 6338), ('Appzxt', 6120), ('AutoNewsAdmin', 5225), ('enre19', 5076), ('piealamodelemonade', 4735)]

score 1156
[('1', 1543648), ('2', 8471), ('3', 3723), ('4', 2099), ('5', 1784), ('6', 1327), ('7', 988), ('8', 805), ('9', 670), ('10', 556)]

title 1176538
[('Dale Wilson', 3445), ('Aria Electra Video Baby Alien Fan Bus Leaked The Fan Van Twitter', 2742), ('Praise Todd Howard', 2626), ('V